In [1]:
import pandas as pd
import numpy as np
import os
import sys
sys.path.insert(0, '../scripts/')

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 196

In [3]:
%load_ext autoreload
%autoreload 1

In [4]:
%aimport etllib
%aimport shapeslib
%aimport smoothlib
%aimport constants

In [5]:
mountain_ptc = pd.read_csv('../precomputed/ddf--datapoints--income_mountain--by--country--year--bracket_200.csv')

In [6]:
mountain_ptc = mountain_ptc.set_index(['country', 'year', 'bracket'])

In [7]:
pop_file = '../../../ddf--gapminder--systema_globalis/countries-etc-datapoints/ddf--datapoints--population_total--by--geo--time.csv'
pop = pd.read_csv(pop_file).set_index(['geo', 'time'])['population_total']

In [8]:
mountain_ptc

income_mountain
country year bracket                 
afg     1800 0           2.386395e-06
             1           2.685107e-06
             2           3.038148e-06
             3           3.438875e-06
             4           3.890295e-06
...                               ...
zwe     2050 195         4.703014e-07
             196         3.880885e-07
             197         3.185998e-07
             198         2.601386e-07
             199         2.112494e-07

[9793000 rows x 1 columns]

In [9]:
pop

geo  time
afg  1800     3280000
     1801     3280000
     1802     3280000
     1803     3280000
     1804     3280000
               ...   
zwe  2096    30940779
     2097    30952208
     2098    30959811
     2099    30964052
     2100    30965421
Name: population_total, Length: 59297, dtype: int64

In [10]:
mountain_num = []

for g, df in mountain_ptc.groupby(['country', 'year']):
    try:
        p = pop.loc[g]
    except KeyError:
        print(f'no population for {g}')
        continue
    df_new = df * p
    df_new['income_mountain'] = np.round(df_new['income_mountain']).astype(int)
    mountain_num.append(df_new)

no population for ('kos', 2000)
no population for ('kos', 2001)
no population for ('kos', 2002)
no population for ('kos', 2003)
no population for ('kos', 2004)
no population for ('kos', 2005)
no population for ('kos', 2006)
no population for ('kos', 2007)
no population for ('kos', 2008)
no population for ('kos', 2009)
no population for ('kos', 2010)
no population for ('kos', 2011)
no population for ('kos', 2012)
no population for ('kos', 2013)
no population for ('kos', 2014)
no population for ('kos', 2015)
no population for ('kos', 2016)
no population for ('kos', 2017)
no population for ('kos', 2018)
no population for ('kos', 2019)


In [11]:
mountain_num = pd.concat(mountain_num)

In [12]:
mountain_num

income_mountain
country year bracket                 
afg     1800 0                      8
             1                      9
             2                     10
             3                     11
             4                     13
...                               ...
zwe     2050 195                   11
             196                    9
             197                    8
             198                    6
             199                    5

[9789000 rows x 1 columns]

In [13]:
# mountain_num.to_csv('../precomputed/ddf--datapoints--income_mountain_numbers--by--country--year--bracket_200.csv')

In [14]:
gs = mountain_ptc.groupby(['country', 'year'])
df = gs.get_group(('ago', 2000))

In [15]:
# gapminder levels
thresholes = [2, 8, 32]
brackets = [etllib.bracket_number_from_income(x) for x in thresholes]

def get_on_rates(ser):
    res = []
    # print(brackets)
    res.append(ser.iloc[:brackets[0]].sum())
    res.append(ser.iloc[brackets[0]:brackets[1]].sum())
    res.append(ser.iloc[brackets[1]:brackets[2]].sum())
    res.append(ser.iloc[brackets[2]:].sum())
    
    res = pd.Series(res)
    res.index.name = 'gm_level'
    return res

In [16]:
get_on_rates(df['income_mountain'])

gm_level
0    0.398455
1    0.482176
2    0.106353
3    0.013015
dtype: float64

In [17]:
gm_rate = gs['income_mountain'].apply(get_on_rates)

In [18]:
gm_rate

country  year  gm_level
afg      1800  0           0.936293
               1           0.060166
               2           0.003185
               3           0.000345
         1801  0           0.936293
                             ...   
zwe      2049  3           0.026921
         2050  0           0.256476
               1           0.553199
               2           0.163469
               3           0.026855
Name: income_mountain, Length: 195860, dtype: float64

In [19]:
gm_num = []

for g, df in gm_rate.groupby(['country', 'year']):
    try:
        p = pop.loc[g]
    except KeyError:
        print(f'no population for {g}')
        continue
    df_new = df * p
    df_new = np.round(df_new).astype(int)
    gm_num.append(df_new)

no population for ('kos', 2000)
no population for ('kos', 2001)
no population for ('kos', 2002)
no population for ('kos', 2003)
no population for ('kos', 2004)
no population for ('kos', 2005)
no population for ('kos', 2006)
no population for ('kos', 2007)
no population for ('kos', 2008)
no population for ('kos', 2009)
no population for ('kos', 2010)
no population for ('kos', 2011)
no population for ('kos', 2012)
no population for ('kos', 2013)
no population for ('kos', 2014)
no population for ('kos', 2015)
no population for ('kos', 2016)
no population for ('kos', 2017)
no population for ('kos', 2018)
no population for ('kos', 2019)


In [20]:
gm_num = pd.concat(gm_num)

In [21]:
gm_num

country  year  gm_level
afg      1800  0            3071040
               1             197345
               2              10446
               3               1132
         1801  0            3071040
                             ...   
zwe      2049  3             637040
         2050  0            6142070
               1           13247970
               2            3914749
               3             643116
Name: income_mountain, Length: 195780, dtype: int64

In [22]:
on_map = {
    0: 'gm_lvl1',
    1: 'gm_lvl2',
    2: 'gm_lvl3',
    3: 'gm_lvl4'
}

on_ents = {
    'gm_lvl1': '<2$/day',
    'gm_lvl2': '2-8$/day',
    'gm_lvl3': '8-32$/day',
    'gm_lvl4': '>32$/day'
}

In [23]:
# new_vals = res1.index.get_level_values('on_level').map(on_map)

gm_rate.index = gm_rate.index.set_levels(list(on_map.values()), level='gm_level')

In [24]:
gm_rate

country  year  gm_level
afg      1800  gm_lvl1     0.936293
               gm_lvl2     0.060166
               gm_lvl3     0.003185
               gm_lvl4     0.000345
         1801  gm_lvl1     0.936293
                             ...   
zwe      2049  gm_lvl4     0.026921
         2050  gm_lvl1     0.256476
               gm_lvl2     0.553199
               gm_lvl3     0.163469
               gm_lvl4     0.026855
Name: income_mountain, Length: 195860, dtype: float64

In [25]:
gm_num.index = gm_num.index.set_levels(list(on_map.values()), level='gm_level')

In [26]:
gm_num

country  year  gm_level
afg      1800  gm_lvl1      3071040
               gm_lvl2       197345
               gm_lvl3        10446
               gm_lvl4         1132
         1801  gm_lvl1      3071040
                             ...   
zwe      2049  gm_lvl4       637040
         2050  gm_lvl1      6142070
               gm_lvl2     13247970
               gm_lvl3      3914749
               gm_lvl4       643116
Name: income_mountain, Length: 195780, dtype: int64

In [27]:
gm_rate.index.names = ['country', 'year', 'gm_level']
gm_rate.name = 'poverty_rate'

In [28]:
from ddf_utils.str import format_float_digits
from functools import partial

In [29]:
formattor = partial(format_float_digits, digits=6)

In [30]:
gm_rate_str = gm_rate.map(formattor)

In [31]:
# !mkdir  ../../poverty_rates

In [32]:
gm_rate_str.to_csv('../../poverty_rates/ddf--datapoints--poverty_rate--by--country--year--gm_level.csv')

In [33]:
gm_num.index.names = ['country', 'year', 'gm_level']
gm_num.name = 'poverty_population'

In [34]:
gm_num.to_csv('../../poverty_rates/ddf--datapoints--poverty_population--by--country--year--gm_level.csv')

In [35]:
# now estimate levels in owd

In [36]:
# we should use the cumsum of income mountain to estimate!

In [37]:
mountain_ptc

income_mountain
country year bracket                 
afg     1800 0           2.386395e-06
             1           2.685107e-06
             2           3.038148e-06
             3           3.438875e-06
             4           3.890295e-06
...                               ...
zwe     2050 195         4.703014e-07
             196         3.880885e-07
             197         3.185998e-07
             198         2.601386e-07
             199         2.112494e-07

[9793000 rows x 1 columns]

In [38]:
cdf = mountain_ptc.groupby(['country', 'year']).cumsum()

In [39]:
cdf

income_mountain
country year bracket                 
afg     1800 0               0.000002
             1               0.000005
             2               0.000008
             3               0.000012
             4               0.000015
...                               ...
zwe     2050 195             0.999998
             196             0.999999
             197             0.999999
             198             0.999999
             199             0.999999

[9793000 rows x 1 columns]

In [40]:
# owid levels
to_estimate = [1.9, 3.2, 5.5, 10]

brackets = [etllib.bracket_number_from_income(x) for x in to_estimate]

brackets_ = [etllib.bracket_number_from_income(x, integer=False) for x in to_estimate]

In [41]:
brackets

[79, 86, 94, 103]

In [42]:
brackets_

[79.25999418556223, 86.78071905112638, 94.59431618637296, 103.21928094887362]

In [43]:
gs = cdf.groupby(['country', 'year'])['income_mountain']

In [44]:
df = gs.get_group(('ago', 2000))

In [45]:
# 79: below 2
# 78: below 1.866066
df.iloc[[78, 79]]

country  year  bracket
ago      2000  78         0.371926
               79         0.398455
Name: income_mountain, dtype: float64

In [46]:
a = df.iloc[78]
b = df.iloc[79]

e = (b - a) * (brackets_[0] - 79)

In [47]:
e + a

0.3788230713035825

In [48]:
# assuming linear function between 2 known brackets.
def get_estimate(df, x):
    b = int(x)
    a = b - 1
    
    ya = df.iloc[a]
    yb = df.iloc[b]
    
    return (yb - ya) * (x - b) + ya

In [49]:
get_estimate(df, brackets_[0])

0.3788230713035825

In [50]:
dec = np.r_[0, [get_estimate(df, x) for x in brackets_], 1]

In [51]:
dec = pd.Series(dec)

In [52]:
dec.shift(-1) - dec

0    0.378823
1    0.214216
2    0.196946
3    0.126553
4    0.083462
5         NaN
dtype: float64

In [53]:
def get_wb_level(ser):
    dec = np.r_[0, [get_estimate(ser, x) for x in brackets_], 1]
    dec = pd.Series(dec)
    res = dec.shift(-1) - dec
    res = res.dropna()
    res.index.name = 'wb_level'
    return res

In [54]:
get_wb_level(df)

wb_level
0    0.378823
1    0.214216
2    0.196946
3    0.126553
4    0.083462
dtype: float64

In [55]:
get_wb_level(df).sum()

1.0

In [56]:
wb_rates = cdf.groupby(['country', 'year'])['income_mountain'].apply(get_wb_level)

In [57]:
wb_rates

country  year  wb_level
afg      1800  0           0.928548
               1           0.048335
               2           0.015658
               3           0.005132
               4           0.002328
                             ...   
zwe      2050  0           0.235782
               1           0.232126
               2           0.226817
               3           0.164502
               4           0.140772
Name: income_mountain, Length: 244825, dtype: float64

In [58]:
wb_map = {
    0: 'wb_lvl1',
    1: 'wb_lvl2',
    2: 'wb_lvl3',
    3: 'wb_lvl4',
    4: 'wb_lvl5'
}

wb_ents = {
    'wb_lvl1': '<1.90$/day',
    'wb_lvl2': '1.9-3.20$/day',
    'wb_lvl3': '3.20-5.50$/day',
    'wb_lvl4': '5.50-10$/day',
    'wb_lvl5': '>10$/day'
}

In [59]:
wb_num = []

for g, df in wb_rates.groupby(['country', 'year']):
    try:
        p = pop.loc[g]
    except KeyError:
        print(f'no population for {g}')
        continue
    df_new = df * p
    df_new = np.round(df_new).astype(int)
    wb_num.append(df_new)

no population for ('kos', 2000)
no population for ('kos', 2001)
no population for ('kos', 2002)
no population for ('kos', 2003)
no population for ('kos', 2004)
no population for ('kos', 2005)
no population for ('kos', 2006)
no population for ('kos', 2007)
no population for ('kos', 2008)
no population for ('kos', 2009)
no population for ('kos', 2010)
no population for ('kos', 2011)
no population for ('kos', 2012)
no population for ('kos', 2013)
no population for ('kos', 2014)
no population for ('kos', 2015)
no population for ('kos', 2016)
no population for ('kos', 2017)
no population for ('kos', 2018)
no population for ('kos', 2019)


In [60]:
wb_num = pd.concat(wb_num)

In [61]:
wb_num

country  year  wb_level
afg      1800  0           3045636
               1            158538
               2             51359
               3             16832
               4              7636
                            ...   
zwe      2050  0           5646500
               1           5558934
               2           5431805
               3           3939476
               4           3371207
Name: income_mountain, Length: 244725, dtype: int64

In [62]:
wb_num.index = wb_num.index.set_levels(list(wb_map.values()), level='wb_level')

In [65]:
wb_num

country  year  wb_level
afg      1800  wb_lvl1     3045636
               wb_lvl2      158538
               wb_lvl3       51359
               wb_lvl4       16832
               wb_lvl5        7636
                            ...   
zwe      2050  wb_lvl1     5646500
               wb_lvl2     5558934
               wb_lvl3     5431805
               wb_lvl4     3939476
               wb_lvl5     3371207
Name: income_mountain, Length: 244725, dtype: int64

In [66]:
wb_num.index.names = ['country', 'year', 'wb_level']
wb_num.name = 'poverty_population'

In [67]:
wb_num.to_csv('../../poverty_rates/ddf--datapoints--poverty_population--by--country--year--wb_level.csv')

In [68]:
wb_rates = wb_rates.map(formattor)

In [69]:
wb_rates

country  year  wb_level
afg      1800  0           0.928548
               1           0.048335
               2           0.015658
               3           0.005132
               4           0.002328
                             ...   
zwe      2050  0           0.235782
               1           0.232126
               2           0.226817
               3           0.164502
               4           0.140772
Name: income_mountain, Length: 244825, dtype: object

In [70]:
wb_rates.index = wb_rates.index.set_levels(list(wb_map.values()), level='wb_level')

In [71]:
wb_rates.index.names = ['country', 'year', 'wb_level']
wb_rates.name = 'poverty_rate'

In [72]:
wb_rates.to_csv('../../poverty_rates/ddf--datapoints--poverty_rate--by--country--year--wb_level.csv')

In [73]:
on_ents

{'gm_lvl1': '<2$/day',
 'gm_lvl2': '2-8$/day',
 'gm_lvl3': '8-32$/day',
 'gm_lvl4': '>32$/day'}

In [74]:
on_ent = pd.DataFrame.from_dict(on_ents, orient='index')
on_ent.index.name = 'gm_level'
on_ent.columns = ['name']

In [75]:
on_ent

,name
gm_level,
gm_lvl1,<2$/day
gm_lvl2,2-8$/day
gm_lvl3,8-32$/day
gm_lvl4,>32$/day


In [76]:
on_ent.to_csv('../../ddf--entities--gm_level.csv')

In [77]:
wb_ent = pd.DataFrame.from_dict(wb_ents, orient='index')
wb_ent.index.name = 'wb_level'
wb_ent.columns = ['name']

In [78]:
wb_ent

,name
wb_level,
wb_lvl1,<1.90$/day
wb_lvl2,1.9-3.20$/day
wb_lvl3,3.20-5.50$/day
wb_lvl4,5.50-10$/day
wb_lvl5,>10$/day


In [79]:
wb_ent.to_csv('../../ddf--entities--wb_level.csv')